In [1]:
print(90)

90


In [2]:
from typing import TypedDict

class UserInfo(TypedDict):
    """A dictionary representing user information."""
    name : str
    age : int

def print_details(user: UserInfo):
    print(f"{user['name']} is {user['age']} years old.")
    
print_details({"name": "Nabeel", "age": 27})

Nabeel is 27 years old.


In [3]:
from pydantic import BaseModel, field_validator
from typing import Optional

class UserInfo(BaseModel):
    name: str
    age: Optional[int] = None
    status: Optional[str] = None

    @field_validator("status", mode="before")
    @classmethod
    def default_status(cls, v, values):
        if v is None:
            age = values.get("age")
            if age is not None and age < 18:
                return "minor"
            return "adult"
        return v
print(UserInfo(name="Zain", age=17))  # status='minor'
print(UserInfo(name="Ayaan"))         # status='adult'


name='Zain' age=17 status=None
name='Ayaan' age=None status=None


In [11]:
from langgraph.graph import StateGraph
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
#from langgraph.prebuilt import add_messages
from langgraph.graph.message import add_messages

In [17]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    tool_result: Optional[str]
    user_name: Optional[str]
    city: Optional[str]

In [24]:
import os
def get_api_key(api_key_name):
    return os.getenv(api_key_name)

In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, google_api_key=get_api_key("GOOGLE_API_KEY"))

In [20]:
## LLM-Based City Extractor

In [21]:
from langchain_core.runnables import RunnableLambda

city_extractor_prompt = (
    "Extract the city name from this question: \"{question}\".\n"
    "If no city is mentioned, respond with 'None'."
)

def extract_city_with_llm(state: State) -> dict:
    question = state["messages"][-1].content
    city_query = city_extractor_prompt.format(question=question)
    response = llm.invoke([("human", city_query)])
    city = response.content.strip().replace(".", "")
    return {"city": None if city.lower() == "none" else city}


In [22]:
## Weather Tool Using State["city"]

In [26]:
import requests, os

OPENWEATHER_API_KEY = get_api_key("OPENWEATHERMAP_API_KEY")

def get_weather(city: str) -> str:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    r = requests.get(url)
    if r.status_code != 200:
        return f"❌ Unable to fetch weather for {city}."
    data = r.json()
    desc = data['weather'][0]['description']
    temp = data['main']['temp']
    return f"☀️ Weather in {city}: {desc}, {temp}°C"

def weather_tool(state: State) -> dict:
    city = state["city"] or "Delhi"
    result = get_weather(city)
    return {"tool_result": result}


In [32]:
from langchain_core.messages import AIMessage

def personalized_response(state: State) -> dict:
    name = state.get("user_name", "friend")
    tool_result = state.get("tool_result", "")
    city = state.get("city", "")
    
    message = f"Hi {name} 👋, here’s the latest update:\n{tool_result}" if tool_result else "Ask me about the weather!"
    return {"messages": [AIMessage(content=message)]}

def name_setter(state: State) -> dict:
    last = state["messages"][-1].content.lower()
    if "my name is" in last:
        name = last.split("my name is")[-1].strip().split()[0]
        return {"user_name": name}
    return {}


In [31]:
def router(state: State) -> str:
    last = state["messages"][-1].content.lower()

    if "my name is" in last:
        return "name_setter"

    if "weather" in last or "temperature" in last:
        return "city_extractor"

    return "llm_response"


In [35]:
workflow = StateGraph(State)

# Nodes
workflow.add_node("router", RunnableLambda(router))
workflow.add_node("name_setter", RunnableLambda(name_setter))
workflow.add_node("city_extractor", RunnableLambda(extract_city_with_llm))
workflow.add_node("weather", RunnableLambda(weather_tool))
workflow.add_node("llm_response", RunnableLambda(personalized_response))

# Routing
workflow.set_entry_point("router")

workflow.add_conditional_edges("router", {
    "name_setter": "name_setter",
    "city_extractor": "city_extractor",
    "llm_response": "llm_response"
})

# After setting name → go to LLM
workflow.add_edge("name_setter", "llm_response")

# After extracting city → get weather → reply
workflow.add_edge("city_extractor", "weather")
workflow.add_edge("weather", "llm_response")

workflow.set_finish_point("llm_response")

# Compile
app = workflow.compile()


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>